In [ ]:
import mlflow
from pyspark.sql import SparkSession
from pyspark.sql.functions import struct
import sys
# dbutils को आयात करें ताकि हम विजेट्स (widgets) का उपयोग कर सकें।
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from mlflow.models.signature import *

# SparkSession को शुरू करें
spark = SparkSession.builder.appName("ModelInference").getOrCreate()

# FIX: मॉडल का नाम train_and_log_model_final.py में उपयोग किए गए नाम से मेल खाना चाहिए
model_name = "workspace.ml.house_price_model_uc"

# dbutils.widgets.get() का उपयोग करें
# यह स्टेज बदलती रहेगी (Staging/Production)
try:
    # `dbutils` का उपयोग करके विजेट से उपनाम (alias) प्राप्त करें
    model_alias = dbutils.widgets.get("alias")
except Exception:
    # यदि नोटबुक को सीधे चलाया जाता है (जॉब के माध्यम से नहीं), तो एक डिफ़ॉल्ट मान सेट करें
    model_alias = "Staging"

print(f"Loading model with alias: {model_alias}")

# मॉडल रजिस्ट्री से मॉडल लोड करना
# FIX: स्टेज के बजाय उपनाम (alias) का उपयोग करें
# Unity Catalog के लिए `databricks-uc` रजिस्ट्री URI का उपयोग करें
model_uri = f"models:/{model_name}@{model_alias}"
model = mlflow.sklearn.load_model(model_uri=model_uri)

print(f"{model_alias} उपनाम से मॉडल सफलतापूर्वक लोड हो गया है।")

# FIX: डेल्टा टेबल से डेटा लोड करने के लिए सीधे Unity Catalog का उपयोग करें
catalog_name = "workspace"
schema_name = "default"
table_name = "house_price_delta"

# पूरी टेबल का नाम बनाएं
full_table_name = f"{catalog_name}.{schema_name}.{table_name}"

# spark.table() का उपयोग करके डेटा को DataFrame में लोड करें
spark_df = spark.read.format("delta").table(full_table_name)
print(f"Successfully loaded data from Unity Catalog table: {full_table_name}")

# मॉडल का उपयोग करके प्रेडिक्शन करना
predictions_df = spark_df.toPandas()
predictions_df['predicted_price'] = model.predict(predictions_df[['sq_feet', 'num_bedrooms', 'num_bathrooms', 'year_built', 'location_score']])

print("प्रेडिक्शन सफलतापूर्वक पूरा हो गया है।")

# प्रेडिक्शन के साथ डेटा दिखाना
display(spark.createDataFrame(predictions_df))
